<a href="https://colab.research.google.com/github/presj/Credit_Risk_Analysis/blob/main/credit_risk_ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving LoanStats_2019Q1.csv to LoanStats_2019Q1 (1).csv


In [2]:
import pandas as pd

In [3]:
import io
df = pd.read_csv(io.BytesIO(uploaded['LoanStats_2019Q1.csv']))
# Dataset is now stored in a Pandas Dataframe

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (0,1,2,3,4,7,13,18,19,24,25,27,28,29,30,31,32,34,36,37,38,39,40,41,42,43,44,46,49,50,51,53,54,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,113,114,115,116,117,118,119,120,121,123,124,125,126,127,128,129,130,131,132,133,134,135,136,138,139,140,141,142,143) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
import numpy as np
import pandas as pd

import imblearn

import sklearn

from collections import Counter
from pathlib import Path

import warnings
warnings.filterwarnings('ignore')

import sys
print(f"{'Python':<20}: {sys.version}")
print(f"{'':<20}: {sys.version_info}")
print(f"{'NumPy':<20}: {np.__version__}")
print(f"{'Pandas':<20}: {pd.__version__}")
print(f"{'Imbalanced-Learn':<20}: {imblearn.__version__}")
print(f"{'Scikit-Learn':<20}: {sklearn.__version__} ")

Python              : 3.7.13 (default, Apr 24 2022, 01:04:09) 
[GCC 7.5.0]
                    : sys.version_info(major=3, minor=7, micro=13, releaselevel='final', serial=0)
NumPy               : 1.21.6
Pandas              : 1.3.5
Imbalanced-Learn    : 0.9.0
Scikit-Learn        : 1.0.2 


In [5]:
pip install imbalanced-learn==0.9

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
import numpy as np
from pathlib import Path
from collections import Counter

In [8]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [9]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [10]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [11]:
# Create our features
X = df.copy()
X = X.drop(columns="loan_status")
X = pd.get_dummies(X)

# Create our target
y = df.loc[:,"loan_status"]

In [12]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.177238,1.0,0.123879,0.876121,0.383161,0.616839,0.860340,0.139660,1.0,1.0
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.381873,0.0,0.329446,0.329446,0.486161,0.486161,0.346637,0.346637,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,1.0,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [13]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# Splitting into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [15]:
# ENSEMBLE LEARNERS
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
brfc = BalancedRandomForestClassifier(n_estimators =100, random_state=1)
balanced = brfc.fit(X_train, y_train)
BalancedRandomForestClassifier()

BalancedRandomForestClassifier()

In [16]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = balanced.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.7885466545953005

In [17]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)

cm_df = pd.DataFrame(cm, index=["Actual High_Risk", "Actual Low_Risk"], columns=["Predicted High_Risk", "Predicted Low_Risk"])
cm_df 


,Predicted High_Risk,Predicted Low_Risk
Actual High_Risk,71,30
Actual Low_Risk,2153,14951


In [18]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.70      0.87      0.06      0.78      0.60       101
   low_risk       1.00      0.87      0.70      0.93      0.78      0.62     17104

avg / total       0.99      0.87      0.70      0.93      0.78      0.62     17205



In [19]:
# List the features sorted in descending order by feature importance
importances = balanced.feature_importances_
sorted(zip(balanced.feature_importances_, X.columns), reverse=True)
importances = sorted(zip(balanced.feature_importances_, X.columns), reverse=True)
for importance in importances:
  print(f'{importance[1]}:  {importance[0]*100:.1f}%')

total_rec_prncp:  7.9%
total_pymnt:  5.9%
total_pymnt_inv:  5.6%
total_rec_int:  5.4%
last_pymnt_amnt:  5.0%
int_rate:  3.0%
issue_d_Jan-2019:  2.1%
installment:  2.0%
dti:  1.7%
out_prncp_inv:  1.7%
total_bal_ex_mort:  1.6%
mths_since_recent_inq:  1.5%
issue_d_Mar-2019:  1.5%
revol_bal:  1.5%
max_bal_bc:  1.5%
out_prncp:  1.5%
mo_sin_old_rev_tl_op:  1.5%
annual_inc:  1.5%
bc_util:  1.5%
tot_hi_cred_lim:  1.4%
mo_sin_old_il_acct:  1.4%
mths_since_rcnt_il:  1.4%
total_rev_hi_lim:  1.3%
avg_cur_bal:  1.3%
bc_open_to_buy:  1.3%
il_util:  1.3%
total_bc_limit:  1.3%
all_util:  1.3%
pct_tl_nvr_dlq:  1.3%
tot_cur_bal:  1.3%
total_il_high_credit_limit:  1.3%
inq_last_12m:  1.3%
total_bal_il:  1.2%
mo_sin_rcnt_tl:  1.1%
loan_amnt:  1.1%
mths_since_recent_bc:  1.1%
num_rev_accts:  1.0%
total_acc:  1.0%
open_acc_6m:  1.0%
num_il_tl:  1.0%
num_actv_rev_tl:  1.0%
mo_sin_rcnt_rev_tl_op:  0.9%
num_bc_tl:  0.9%
num_sats:  0.9%
num_op_rev_tl:  0.8%
num_bc_sats:  0.8%
num_tl_op_past_12m:  0.8%
open_act_

In [20]:
#AdaBoost
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier
EE = EasyEnsembleClassifier(n_estimators=100, random_state=1)
EE_trained = EE.fit(X_train, y_train)
EasyEnsembleClassifier()

EasyEnsembleClassifier()

In [21]:
# Calculated the balanced accuracy score
y_pred = EE_trained.predict(X_test)
balanced_accuracy_score(y_test,y_pred)

0.9316600714093861

In [22]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)

cm_df = pd.DataFrame(cm, index=["Actual High_Risk", "Actual Low_Risk"], columns=["Predicted High_Risk", "Predicted Low_Risk"])
cm_df 

,Predicted High_Risk,Predicted Low_Risk
Actual High_Risk,93,8
Actual Low_Risk,983,16121


In [23]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.09      0.92      0.94      0.16      0.93      0.87       101
   low_risk       1.00      0.94      0.92      0.97      0.93      0.87     17104

avg / total       0.99      0.94      0.92      0.97      0.93      0.87     17205

